In [1]:
# import all necessary
from Bio import SeqIO
import numpy as np
from scipy.spatial.distance import pdist, cdist, squareform
import matplotlib.pyplot as plt
import random

In [2]:
# read the fasta file
sequences = []
for item in SeqIO.parse("HW2.fas", "fasta"):
    # turn sequences into lists of integers
    sequences.append([ord(x) for x in list(item.seq)])

In [3]:
# print how many sequences we have
print(len(sequences))

# check if all sequences have the same length
# print the length if yes
if len({len(seq) for seq in sequences}) == 1:
    print(len(sequences[0]))

120
264


In [4]:
# find normalized (divided by the length of a sequence) pairwise Hamming distances
matrix_of_normalized_hamming_distances = squareform(pdist(sequences, 'hamming'))

# turn them into Hamming distances
hamming_distance_matrix = (len(sequences[0])*matrix_of_normalized_hamming_distances).astype(int)

In [ ]:
from sklearn.manifold import MDS
mds = MDS(n_components = 2, dissimilarity = "precomputed", random_state = 0)

similarities = mds.fit_transform(hamming_distance_matrix)

In [ ]:
plt.scatter(similarities[:, 0], similarities[:, 1], marker = "x")

In [ ]:
# INPUT: 
#   points <- data points (as a list of lists)
#   k      <- number of clusters (as an integer)

# OUTPUT: 
#   a list of labels that idetify to which cluster a point belongs   

def kmeans_from_scratch(points, k = 5):

    centroids_indeces = random.sample(range(len(points)), k)
    centroids = [points[i] for i in centroids_indeces]

    # for each point, we find distances between centrods and the point
    # the result is a marrix of the size 120 rows by k columns
    distances_to_centroids = cdist(points, centroids, 'euclidean')

    # we label all points as 0,1,2,... depending on their closeness to a centroid
    # axis = 1 as we go each time along a row,
    # and choose an index at which minimum is achieved in the row
    labels = np.argmin(distances_to_centroids, axis = 1)

    while True:
        # we calculate clusters' means
        cluster_means = [0] * k
        for the_label in range(k):
            points_labeled_with_the_label = [element for count, element in enumerate(points) if labels[count] == the_label]
            cluster_means[the_label] = np.mean(points_labeled_with_the_label, axis = 0) 

        # now we use cluster_means as centers of the clusters
        # recalculate distances to the clusters' centers
        # and relabel point if needed
        centroids_new = [item.tolist() for item in cluster_means]
        distances_to_centroids_new = cdist(points, centroids_new, 'euclidean')
        labels_new = np.argmin(distances_to_centroids_new, axis = 1)
        if (labels_new == labels).all():
            return(labels)
            break
        else:
            labels = labels_new

In [ ]:
points = [item.tolist() for item in similarities]
labels = kmeans_from_scratch(points, k = 5)    
print(labels)    

In [ ]:
plt.scatter(similarities[:, 0], similarities[:, 1], marker = "x", c = labels)